In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
os.chdir('/Users/dapoadegbile/estimating-impact-of-opioids-2020-team2')
os.getcwd()

'/Users/dapoadegbile/estimating-impact-of-opioids-2020-team2'

In [3]:
pa_pre_policy = pd.read_csv("./20_intermediate_files/PA_CountyPop_2000to2009.csv", header = [3])
pa_post_policy = pd.read_csv("./20_intermediate_files/PA_CountyPop_2010to2019.csv", header = [3])

In [4]:
def clean_pop(df1,df2):
    df1 = df1.drop(["Unnamed: 1", "2000", "2001", "2002", "Unnamed: 12", "Unnamed: 13"], axis=1)
    df1 = df1.rename(columns={"Unnamed: 0": "County"})
    df2 = df2.drop(["Census", "Estimates Base", "2016", "2017", "2018", "2019"], axis=1)
    df2 = df2.rename(columns={"Unnamed: 0": "County"})
    df1.drop(df1.head(1).index, inplace=True)
    df1.drop(df1.tail(8).index, inplace=True)
    df2.drop(df2.head(1).index, inplace=True)
    df2.drop(df2.tail(5).index, inplace=True)
    df1["County"] = df1["County"].str[1:]
    df2["County"] = df2["County"].str[1:]
    df1 = df1.melt(id_vars=["County"])
    df1 = df1.rename(columns={"variable": "Year", "value": "Population"})
    df1 = df1.groupby(["Year", "County"], as_index=False).sum()
    df2 = df2.melt(id_vars=["County"])
    df2 = df2.rename(columns={"variable": "Year", "value": "Population"})
    df2 = df2.groupby(["Year", "County"], as_index=False).sum()
    df2[["County1", "State"]] = df2.County.str.split(", ", expand=True)
    df2 = df2.drop(["County", "State"], axis=1)
    df2 = df2.rename(columns={"County1": "County"})
    df_concat = pd.concat([df1,df2],ignore_index=True)
    return df_concat


In [5]:
PA_total_pop = clean_pop(pa_pre_policy, pa_post_policy)

In [6]:
PA_total_pop

,Year,County,Population
0,2003,Adams County,"95,503"
1,2003,Allegheny County,"1,258,420"
2,2003,Armstrong County,"71,645"
3,2003,Beaver County,"177,523"
4,2003,Bedford County,"49,812"
...,...,...,...
866,2015,Washington County,"207,718"
867,2015,Wayne County,"51,823"
868,2015,Westmoreland County,"356,923"
869,2015,Wyoming County,"27,812"


## Load in PA Shipment Data


In [7]:
use_cols = ["BUYER_STATE","BUYER_COUNTY", "CALC_BASE_WT_IN_GM", "MME_Conversion_Factor","TRANSACTION_DATE", "DRUG_NAME"]

pa_shipment = pd.read_csv('./20_intermediate_files/arcos-pa-statewide-itemized.csv.gz',  compression='gzip', usecols = use_cols,
                chunksize = 1000000,
                error_bad_lines=False)


In [8]:
df1 = pd.DataFrame()

for i in pa_shipment:
    a = pd.DataFrame(i)
    df1 = df1.append(a)

In [9]:
df1

,BUYER_STATE,BUYER_COUNTY,DRUG_NAME,TRANSACTION_DATE,CALC_BASE_WT_IN_GM,MME_Conversion_Factor
0,PA,DELAWARE,OXYCODONE,3142013,7.172000,1.5
1,PA,DELAWARE,OXYCODONE,9042013,24.205500,1.5
2,PA,NORTHUMBERLAND,HYDROCODONE,5012006,0.136215,1.0
3,PA,NORTHUMBERLAND,HYDROCODONE,1252007,0.454050,1.0
4,PA,SUSQUEHANNA,HYDROCODONE,2162006,13.621500,1.0
...,...,...,...,...,...,...
10651693,PA,BUCKS,OXYCODONE,11152010,5.379000,1.5
10651694,PA,PHILADELPHIA,OXYCODONE,8032009,1.793000,1.5
10651695,PA,PHILADELPHIA,OXYCODONE,11092009,8.068500,1.5
10651696,PA,PHILADELPHIA,OXYCODONE,11162010,4.482500,1.5


In [10]:
df1['TRANSACTION_YEAR'] = df1['TRANSACTION_DATE'].astype(str).str[-4:] #get the year from this column
        
df1['CALC_BASE_WT_IN_MG'] = df1['CALC_BASE_WT_IN_GM'] * 1000 # convert to milligrams

df1['MORPHINE_EQUIV_IN_MG'] = df1['CALC_BASE_WT_IN_MG'] * df1['MME_Conversion_Factor']



final_shipments_PA = df1[["TRANSACTION_YEAR", "BUYER_STATE", "BUYER_COUNTY", "MORPHINE_EQUIV_IN_MG"]]

final_shipments_PA = final_shipments_PA.rename(columns={"BUYER_COUNTY": "County", "TRANSACTION_YEAR":"Year"})

final_shipments_PA

,Year,BUYER_STATE,County,MORPHINE_EQUIV_IN_MG
0,2013,PA,DELAWARE,10758.000
1,2013,PA,DELAWARE,36308.250
2,2006,PA,NORTHUMBERLAND,136.215
3,2007,PA,NORTHUMBERLAND,454.050
4,2006,PA,SUSQUEHANNA,13621.500
...,...,...,...,...
10651693,2010,PA,BUCKS,8068.500
10651694,2009,PA,PHILADELPHIA,2689.500
10651695,2009,PA,PHILADELPHIA,12102.750
10651696,2010,PA,PHILADELPHIA,6723.750


In [11]:
final_shipments_PA[["Year"]] = final_shipments_PA["Year"].astype(int)

In [12]:
PA_total_pop


,Year,County,Population
0,2003,Adams County,"95,503"
1,2003,Allegheny County,"1,258,420"
2,2003,Armstrong County,"71,645"
3,2003,Beaver County,"177,523"
4,2003,Bedford County,"49,812"
...,...,...,...
866,2015,Washington County,"207,718"
867,2015,Wayne County,"51,823"
868,2015,Westmoreland County,"356,923"
869,2015,Wyoming County,"27,812"


In [13]:
PA_total_pop["County"] = PA_total_pop["County"].str.upper() #make entire column upper case 
PA_total_pop['County'] = PA_total_pop['County'].astype(str).str[:-7] # remove "county" from county name 

In [14]:
PA_total_pop

,Year,County,Population
0,2003,ADAMS,"95,503"
1,2003,ALLEGHENY,"1,258,420"
2,2003,ARMSTRONG,"71,645"
3,2003,BEAVER,"177,523"
4,2003,BEDFORD,"49,812"
...,...,...,...
866,2015,WASHINGTON,"207,718"
867,2015,WAYNE,"51,823"
868,2015,WESTMORELAND,"356,923"
869,2015,WYOMING,"27,812"


In [55]:
final_shipments_PA['County'] = final_shipments_PA['County'].astype(str)
PA_total_pop['County'] = PA_total_pop['County'].astype(str)

PA_total_pop['Year'] = PA_total_pop['Year'].astype(int)
final_shipments_PA['Year'] = final_shipments_PA['Year'].astype(int)

# pd.merge(final_shipments_PA, PA_total_pop, on= "County")
# mergeddf = final_shipments_PA.merge(PA_total_pop, how= 'left')

# mergeddf = final_shipments_PA.merge(PA_total_pop, on= ["Year", "County"],  how='left')

In [57]:
mergeddf = final_shipments_PA.merge(PA_total_pop, on= ["Year", "County"],  how='left')
mergeddf

,Year,BUYER_STATE,County,MORPHINE_EQUIV_IN_MG,Population
0,2013,PA,DELAWARE,10758.000,"561,499"
1,2013,PA,DELAWARE,36308.250,"561,499"
2,2006,PA,NORTHUMBERLAND,136.215,"93,475"
3,2007,PA,NORTHUMBERLAND,454.050,"93,728"
4,2006,PA,SUSQUEHANNA,13621.500,"43,376"
...,...,...,...,...,...
10651693,2010,PA,BUCKS,8068.500,"625,385"
10651694,2009,PA,PHILADELPHIA,2689.500,"1,514,694"
10651695,2009,PA,PHILADELPHIA,12102.750,"1,514,694"
10651696,2010,PA,PHILADELPHIA,6723.750,"1,528,283"


In [58]:
mergeddf.to_csv(r"/Users/dapoadegbile/estimating-impact-of-opioids-2020-team2/20_intermediate_files/PA_shipment_and_pop.csv")